In [6]:
import os
import requests
import json
import csv
import pandas as pd
#conda install -c conda-forge python-dotenv
from dotenv import load_dotenv
from time import sleep
load_dotenv()

True

In [7]:

PATH_TO_DMCBH_MEMBERS_CSV = "Copy of DMCBH member IRP categories_Aug2023.csv"
API_KEY = os.getenv("API_KEY")
header = {
    "X-ELS-APIKey": API_KEY,
}
api_url = "http://api.elsevier.com/content/search/author?"

In [8]:
def get_api_json(params: dict):
    try:
        response = requests.get(url=api_url, params=params, headers=header)
        response.raise_for_status
        data = json.loads(response.text)
    except requests.exceptions.HTTPError as e:
        print("HTTPS error: ", e)
    except requests.exceptions.RetryError as e:
        print("Max retries exceeded: ", e)
    return data

In [9]:
member_arr = []
with open(PATH_TO_DMCBH_MEMBERS_CSV, "r") as f:
    r = csv.reader(f)
    for row in r:
        if row[0] != "":
            member_arr.append(row)

# skip header rows
member_arr = member_arr[2:]
print(len(member_arr))

# columns of the DMCBH CSV file
cols = ['Last Name','First Name','Member Type','Mental Health & Addictions','Brain Development & Neurodevelopmental Disorders','Learning Memory & Dementias','Sensory Motor Systems & Movement Disorders','Brain Injury & Repair','Google Scholar?','GS Link']

last_names = []
first_names = []
affils = []
auth_ids = []
orcids = []
tags = []

warnings = []
subjects = []

no_search_results = []

affiliation_list = ["The University of British Columbia", "Djavad Mowafaghian Centre for Brain Health", "BC Children's Hospital Research Institute"]

129


In [11]:
for member in member_arr:
    sleep(1)
    lastname = member[0].strip()
    firstname = member[1].strip()

    print("searching for " + firstname + " " + lastname)

    query = f"authlastname({lastname}) and authfirst({firstname})"
    params = {
        "query":query
    }

    data = get_api_json(params)

    warning = ""

    num_authors_found = int(data["search-results"]['opensearch:totalResults'])
    if num_authors_found == 0:
        print("no scival profile found for " + firstname + " " + lastname + "\n")
        no_search_results.append(member)
        continue
    if num_authors_found > 1:
        print("more than 1 profile found for " + firstname + " " + lastname)
        warning = warning + "multiple profiles; "
    try:
        affil_name = data["search-results"]["entry"][0]["affiliation-current"]["affiliation-name"]
        if affil_name not in affiliation_list:
            warning = warning + "not affiliated with UBC"
    except:
        warning = "no affiliation"
        affil_name = ""

    author_id = data["search-results"]["entry"][0]["dc:identifier"].split(":")[1]
    # affil_id = data["search-results"]["entry"][0]["affiliation-current"]["affiliation-id"]

    subject = ""
    try:
        for i in data["search-results"]["entry"][0]["subject-area"]:
            subject = f'{i["$"]} - {i["@frequency"]} publications; ' + subject
    except:
        pass

    orcid = ""
    try:
        orcid = data["search-results"]["entry"][0]["orcid"]
    except:
        pass
    
    #find primary IRP
    for i in range(3,8):
        if member[i].strip() == "Primary":
            tags.append(cols[i])

    last_names.append(lastname)
    first_names.append(firstname)
    affils.append(affil_name)
    auth_ids.append(author_id)
    warnings.append(warning)
    subjects.append(subject)
    orcids.append(orcid)

    print("\n")

searching for Khaled Abd-Elrahman


searching for Douglas Allan
more than 1 profile found for Douglas Allan


searching for Doug Altshuler
no scival profile found for Doug Altshuler

searching for Vanessa Auld


searching for Jehannine Austin


searching for Shelina Babul
more than 1 profile found for Shelina Babul


searching for Shernaz Bamji


searching for Phil Barker
more than 1 profile found for Phil Barker


searching for Steven Barnes
more than 1 profile found for Steven Barnes


searching for Alasdair Barr


searching for Jason Barton
more than 1 profile found for Jason Barton


searching for Clare Beasley


searching for Lara Boyd
more than 1 profile found for Lara Boyd


searching for Neil Cashman
more than 1 profile found for Neil Cashman


searching for Mark Cembrowski


searching for Trisha Chakrabarty


searching for Annie Ciernia
more than 1 profile found for Annie Ciernia


searching for Luke Clark
more than 1 profile found for Luke Clark


searching for Ann-Marie Crai

In [12]:
out_dict = {
    "Last Name": last_names,
    "First Name": first_names,
    "Affiliation": affils,
    "Scopus Author ID": auth_ids,
    "ORCID": orcids,
    "Tags": tags,
    "Research Areas": subjects,
    "Warnings": warnings,
}
print(no_search_results)

[['Altshuler', 'Doug', 'Full', '', '', '', 'Primary', '', '1', '', '', 'https://scholar.google.com/citations?hl=en&user=EVACejsAAAAJ'], ['Altshuler', 'Doug', 'Full', '', '', '', 'Primary', '', '1', '', '', 'https://scholar.google.com/citations?hl=en&user=EVACejsAAAAJ']]


In [13]:
df = pd.DataFrame.from_dict(out_dict)
df.to_csv("DMCBH_scival_input1.csv")

In [9]:
lastname = "Auld"
firstname = "Vanessa"
query = f"authlastname({lastname}) and authfirst({firstname})"
params = {
    "query":query
}
data = get_api_json(params)

In [10]:
print(data)

{'search-results': {'opensearch:totalResults': '1', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '1', 'opensearch:Query': {'@role': 'request', '@searchTerms': 'authlastname(Auld) and authfirst(Vanessa)', '@startPage': '0'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/author?start=0&count=25&query=authlastname%28Auld%29+and+authfirst%28Vanessa%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/author?start=0&count=25&query=authlastname%28Auld%29+and+authfirst%28Vanessa%29', '@type': 'application/json'}], 'entry': [{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/author/author_id/7004622834'}, {'@_fa': 'true', '@ref': 'search', '@href': 'https://api.elsevier.com/content/search/author?query=au-id%287004622834%29'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/author/citedby.uri?pa

In [12]:
print(data["search-results"]["entry"][0])
print(data["search-results"]["entry"][0]["dc:identifier"].split(":")[1])
print(data["search-results"]["entry"][0]["affiliation-current"])
print(data["search-results"]["entry"][0]["subject-area"])



{'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/author/author_id/7004622834'}, {'@_fa': 'true', '@ref': 'search', '@href': 'https://api.elsevier.com/content/search/author?query=au-id%287004622834%29'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/author/citedby.uri?partnerID=HzOxMe3b&citedAuthorId=7004622834&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-author', '@href': 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=7004622834&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/author/author_id/7004622834', 'dc:identifier': 'AUTHOR_ID:7004622834', 'eid': '9-s2.0-7004622834', 'orcid': '0000-0003-3976-9186', 'preferred-name': {'surname': 'Auld', 'given-name': 'Vanessa J.', 'initials': 'V.J.'}, 'name-variant': [{'@_fa': 'true', 'surname': 'Auld', 'given-name': 'V.', 'initials': 'V.'}, {'@_fa': 'true', 'surname': 'Auld', 'given-name': 'Vanessa', 'initials': 'V.'}, {

In [34]:
#UBC "affiliation-id":"60010365"

num_authors_found = data["search-results"]["opensearch:totalResults"]

if num_authors_found == 0:
    print("no author found")
if num_authors_found > 1:
    print("more than one author found")
    #find author with matching affiliation id

1


In [ ]:
#add name, institution, etc to csv. tagged in diff irp groups?
